### Imports

In [24]:
import json
import os
import requests
from bs4 import BeautifulSoup

### Utils

In [30]:
def extract(ancestor, selector, attribute = None, multiple = False):
    if multiple:
        if attribute:
            return [tag[attribute] for tag in ancestor.select(selector)]
        return [tag.get_text() for tag in ancestor.select(selector)]
    if attribute:
        try:
            return ancestor.select_one(selector)[attribute]
        except TypeError:
            return None
    try:
        return ancestor.select_one(selector).get_text()
    except AttributeError:
        return None

### Extraction of opinions

In [25]:
with open("cookie.json","r") as jsonFile:
    headers = json.load(jsonFile)
    

In [ ]:
productID = "84514582"
next_page = f"https://www.ceneo.pl/{productID}#tab=reviews_scroll"
allOpinions = []
while next_page:
    response = requests.get(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            singleOpinion = {}
            singleOpinion["opinion_id"] = opinion["data-entry-id"]
            singleOpinion["author"] = extract(opinion, "span.user-post__author-name")
            singleOpinion["recommendation"] = extract(opinion, "span.user-post__author-recomendation > em")
            singleOpinion["stars"] = extract(opinion,"span.user-post__score-count")
            singleOpinion["content"] = extract(opinion,"div.user-post__text")
            singleOpinion["pros"] = extract(opinion,"div.review-feature__item--positive",None,True)
            singleOpinion["cons"] = extract(opinion,"div.review-feature__item--negative",None,True)
            singleOpinion["vote_yes"] = extract(opinion, "button.vote-yes", "data-total-vote")
            singleOpinion["vote_no"] = extract(opinion, "button.vote-no", "data-total-vote")
            singleOpinion["published"] = extract(opinion,"span.user-post__published > time:nth-child(1)","datetime")
            singleOpinion["purchased"] = extract(opinion,"span.user-post__published > time:nth-child(2)", "datetime")
            allOpinions.append(singleOpinion)
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None
    else:
        print("Smth went wrong")
print(allOpinions)

[{'opinion_id': '12535429', 'author': '\ng...6\n', 'recommendation': None, 'stars': '3/5', 'content': 'Instalacja łatwa. Oprogramowanie HP Smart bezproblemowe. Zwrot 100zl Cashback z Promocji po kilku dniach od zakupu. Skanuje. Drukuje. \nPrzez pierwsze 2-3 dni wszystko w porządku, wydruki czarne i kolorowe bardzo dobre. Po 3 dniu, zupełnie nagle, drukarka zaczyna drukować białe poziome kreski na wydrukach kolorowych. W czarnym bardzo dobrze. Wydruki w najlepszej jakości są bardzo dobre. Wydruki w jakości normalnej i roboczej są moim zdaniem wadliwe. Kupiłem 2 drukarki. Jedną udało mi się zwrócić w ramach 14 zwrotu. Druga niestety została. \nProducent HP raz uznał gwarancję. HP przysłałi nową głowicę kolorową, a jak problem nie został rozwiązany, całą nową drukarkę. Trzecia drukarka tak samo, drukuje białe kreski na wydrukach kolorowych. \nZgłoszone ponownie do HP, tym razem w ocenie wsparcia HP, drukarki atramentowe tak drukują... Pozostawię to bez komentarza. Chciałem zgłosić w ramac

In [27]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
    
with open(f"./opinions/{productID}.json","w",encoding="UTF-8") as jsonFile:
    json.dump(allOpinions, jsonFile, ensure_ascii = False, indent = 4)